In [129]:
import pandas as pd
import folium
from folium import plugins

In [130]:
df = pd.read_excel('Punto Limpio RM.xlsx', sheet_name='Hoja1')


In [131]:
df["lat_num"] = (
    df["Latitud"]
    .astype(str)
    .str.replace(",", ".", regex=False)
    .str.strip()
)

df["lon_num"] = (
    df["Longitud"]
    .astype(str)
    .str.replace(",", ".", regex=False)
    .str.strip()
)

df["lat_num"] = pd.to_numeric(df["lat_num"], errors="coerce")
df["lon_num"] = pd.to_numeric(df["lon_num"], errors="coerce")


In [132]:
mapa = folium.Map(
    location=[-33.45, -70.65],
    zoom_start=12,
    tiles="CartoDB positron"
)


In [133]:
def popup_html(row):
    raee = row.get("Residuos RAEE", "")
    tipos = row.get("Tipos", "")

    return f"""
    <div style="font-family: Arial; font-size: 13px; line-height: 1.35; width: 260px;">
        <h4 style="color: #2E7D32; margin-bottom: 10px;">
            ♻️ {row["Nombre"]}
        </h4>
      <div style="margin-bottom: 6px;">
        <b>RAEE:</b> {raee}
      </div>

      <div>
        <b>Tipos aceptados:</b><br>
        <span style="color:#333;">{tipos}</span>
      </div>
    </div>
    """


In [134]:
for _, row in df.iterrows():

    # color según RAEE
    if row["Residuos RAEE"] == "Sí":
        color = "#2E7D32"   # verde oscuro
    else:
        color = "#4173E7"   # verde claro
    
    # Crear marcador con icono personalizado
    folium.CircleMarker(
        location=[row["lat_num"], row["lon_num"]],
        radius=10,
        tooltip=f"<b>{row['Nombre']}</b>",
        popup=folium.Popup(popup_html(row), max_width=250),
        color=color,           # Borde más oscuro
        fill=True,
        fillColor=color,       # Verde más vibrante
        fill_opacity=0.8,
        weight=3,                  # Grosor del borde
    ).add_to(mapa)


# 4. Añadir controles adicionales
plugins.Fullscreen(position='topleft').add_to(mapa)
plugins.LocateControl().add_to(mapa)

# 5. Ajustar bounds con padding
mapa.fit_bounds(
    [[df["lat_num"].min(), df["lon_num"].min()],
     [df["lat_num"].max(), df["lon_num"].max()]],
    padding=[50, 50]  # Añade margen
)

# 6. Añadir título personalizado
title_html = '''
<div style="position: fixed; 
            top: 10px; left: 50px; width: 300px; height: 60px; 
            background-color: white; border:2px solid #2E7D32; z-index:9999; 
            font-size:16px; padding: 10px; border-radius: 5px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.2);">
    <h4 style="margin: 0; color: #2E7D32;">
        ♻️ Puntos Limpios - Santiago
    </h4>
    <p style="margin: 5px 0; font-size: 12px; color: #666;">
        {} ubicaciones disponibles
    </p>
</div>
'''.format(len(df))

mapa.get_root().html.add_child(folium.Element(title_html))

legend_html = """
<div style="
position: fixed;
bottom: 40px;
left: 40px;
width: 220px;
background-color: white;
border: 2px solid grey;
z-index:9999;
font-size:14px;
padding: 10px;
">
<b>Convenciones</b><br>
<span style="color:#2E7D32;">●</span> Acepta RAEE<br>
<span style="color:#4173E7;">●</span> No acepta RAEE
</div>
"""

mapa.get_root().html.add_child(folium.Element(legend_html))


mapa.save("index.html")


